In [1]:
%load_ext autoreload
%autoreload 2

import tensordict
from tensordict import TensorDict
import torch
import torchrl

from tqdm.auto import tqdm

from src.test_config import create_batched_test_one_stage_supplier_network
from src.batched_base_config import create_batched_base_supplier_network
from src.batched_env.env_wrapper import MaxDStrategy, DefaultStrategy, NearestSufficientChangeDStrategy
from src.model_zoo.my_ppo.ppo import ppo

In [2]:
from src.model_zoo.my_ppo.actor_critic_models.mlp_actor_critic import MLPActorCriticFabric
from src.model_zoo.my_ppo.actor_critic_models.mlp_w_additional_features import MLPwFActorCriticFabric

In [3]:
ac = ppo(
    lambda : create_batched_base_supplier_network(1000, MaxDStrategy(), 'cuda'),
    actor_critic=MLPwFActorCriticFabric(), 
    steps_per_epoch=17000, 
    gamma=1.0, 
    clip_ratio=0.1, 
    target_kl=0.01, 
    pi_lr=1e-4, 
    vf_lr=5e-4,
    train_pi_iters=40, 
    train_v_iters=20, 
    epochs=500, 
    lam=0.2, 
    device='cuda',
)

  0%|          | 0/17 [00:00<?, ?it/s]

wandb: Currently logged in as: vjlad (valoebi). Use `wandb login --relogin` to force relogin


pi_iter:   0%|          | 0/40 [00:00<?, ?it/s]

v_iter:   0%|          | 0/20 [00:00<?, ?it/s]

playing:   0%|          | 0/17000 [00:00<?, ?it/s]

epoch:   0%|          | 0/500 [00:00<?, ?it/s]

Traceback (most recent call last):
  File "D:\maga\transportation_problem\rl-for-transportation-problem\src\model_zoo\my_ppo\ppo.py", line 183, in ppo
    buf.sample_new_trajectories(ac, tqdm_playing)
  File "D:\maga\transportation_problem\rl-for-transportation-problem\src\model_zoo\my_ppo\buffering.py", line 126, in sample_new_trajectories
    self.finalize_trajectory(tensor_ids, last_v, log_ep_ret)
  File "D:\maga\transportation_problem\rl-for-transportation-problem\src\model_zoo\my_ppo\buffering.py", line 89, in finalize_trajectory
    self.ret_buf[path_slice] = discounted_cumsum_right(rews[None, ...], self.gamma)[0, :-1]
  File "D:\maga\.torch_env\lib\site-packages\torch_discounted_cumsum\discounted_cumsum.py", line 120, in discounted_cumsum_right
    gamma = torch.tensor(gamma).to(input)
KeyboardInterrupt


ClipFrac,██▅▇▇▅█▇▅▇▇▅▇▇▅▇▇▅▄▃▂▁▁▁▃▁▁▃▃▂▃▃▁▂▃▃▂▂▂▃
Entropy,████████▇▇▇▇▇▆▆▆▆▆▄▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
EpLen_Max,▅▅▅█▅█▅▅▁▁▁▅▁▅▁▁▁▁▁▁▁▁▁▁▅▁▅▁▁▁▁▅▁▅▅▁▅▅▁▁
EpLen_Mean,▁▁▂▄▅▆█▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
EpLen_Min,▁▁▁▁▁▁▁▁▁▁▁▁▁▁█████████████████▁█████▁██
EpLen_Std,████▇▇▆▄▃▂▁▂▁▁▁▂▁▂▁▁▁▁▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
EpRet_Max,▁▁▅▂▄▅█▇▇▆▆▅▄▄▅▆▅▅▃▃▅▃▅▄▅▄▄▆▅▅▆▅▇▅▅▅▄▄▄▄
EpRet_Mean,▁▁▂▂▃▄▆▆▇▇▇▇▇▇▇▇▇▇▇▇████████████████████
EpRet_Min,▁▁▁▁▁▃▃▅▄▇▆▇▆▇▆▆▇▆▇▆█▇▇▇█▆▆▇▇▇▇▇▇▆▇▆▇▆▇▇
EpRet_Std,█████▇▆▄▃▂▁▂▁▁▂▁▂▂▂▂▁▁▁▁▁▁▂▂▂▂▂▂▁▂▂▂▂▂▂▂
Epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


KeyboardInterrupt: 

In [25]:
len(create_batched_base_supplier_network(10, MaxDStrategy()).reset()['state'].keys())

2

In [4]:
env = create_batched_base_supplier_network(20, MaxDStrategy())
MLPwFActorCriticFabric()(env)

ActorCritic(
  (pi): Actor(
    (model): MlpWithFeaturesModel(
      (by_layer_models): ModuleList(
        (0): Sequential(
          (0): Flatten(start_dim=-3, end_dim=-1)
          (1): FeatureExtractor()
          (2): Linear(in_features=108, out_features=256, bias=True)
          (3): LeakyReLU(negative_slope=0.01)
          (4): Linear(in_features=256, out_features=256, bias=True)
          (5): LeakyReLU(negative_slope=0.01)
        )
        (1): Sequential(
          (0): Flatten(start_dim=-3, end_dim=-1)
          (1): FeatureExtractor()
          (2): Linear(in_features=162, out_features=256, bias=True)
          (3): LeakyReLU(negative_slope=0.01)
          (4): Linear(in_features=256, out_features=256, bias=True)
          (5): LeakyReLU(negative_slope=0.01)
        )
      )
      (joint_body): Sequential(
        (0): Linear(in_features=512, out_features=512, bias=True)
        (1): LeakyReLU(negative_slope=0.01)
        (2): Linear(in_features=512, out_features=648, bia

In [18]:
a, v, logp = ac.step(env.reset())
a,v,logp

(tensor([516, 438, 281,  54, 513, 509,  61, 432, 353, 314]),
 tensor([-0.0432, -0.0432, -0.0432, -0.0432, -0.0432, -0.0432, -0.0432, -0.0432,
         -0.0432, -0.0432]),
 tensor([-6.4003, -6.4369, -6.4792, -6.4997, -6.4535, -6.5182, -6.5120, -6.4478,
         -6.4365, -6.4481]))

In [19]:
new_s, r, done =  env.step(a)
new_s, r, done

(TensorDict(
     fields={
         state: TensorDict(
             fields={
                 0: Tensor(shape=torch.Size([10, 4, 3, 3]), device=cpu, dtype=torch.float32, is_shared=False),
                 1: Tensor(shape=torch.Size([10, 3, 3, 6]), device=cpu, dtype=torch.float32, is_shared=False)},
             batch_size=torch.Size([10]),
             device=None,
             is_shared=False),
         valid_actions_mask: Tensor(shape=torch.Size([10, 4, 3, 3, 3, 6]), device=cpu, dtype=torch.bool, is_shared=False)},
     batch_size=torch.Size([10]),
     device=None,
     is_shared=False),
 tensor([ -862.,  -744., -4351.,  -911.,  -715., -3087., -2186.,  -732., -4096.,
          -780.]),
 tensor([False, False, False, False, False, False, False, False, False, False]))

In [20]:
ac.step(new_s)

(tensor([209, 531, 152, 409, 208, 229,  40, 101, 580,  25]),
 tensor([-0.0711, -0.1535, -1.7211, -0.1031, -0.0744, -0.0923, -0.0495, -0.4188,
         -1.8136,  0.0488]),
 tensor([-6.7919, -6.5528, -7.2161, -6.6751, -6.6455, -5.2973, -4.8860, -5.5772,
         -3.8028, -6.1535]))

In [21]:
y = env.reset()

In [22]:
len(x), len(y)

(10, 20)

In [25]:
y[torch.arange(20) < 0] = x[torch.arange(10) < 0]

tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
        False, False, False, False, False, False, False, False, False, False])